In [95]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.0993509292602539  seconds 


In [7]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [8]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [9]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [10]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [96]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [97]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [98]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [99]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [100]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [101]:
remove_dashes(data)

In [102]:
df=pd.DataFrame(data)

In [103]:
X=df[['age','cleaned_stop_words']]

In [104]:
dummy_variable_1 = pd.get_dummies(df["category"])
X=pd.concat([X,dummy_variable_1], axis=1)

In [105]:
del dummy_variable_1

In [106]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [107]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(X['cleaned_stop_words']).toarray()

In [25]:
# vectorizer = CountVectorizer()
# vectorizer.fit(X['cleaned_stop_words'])
# posts = vectorizer.transform(X['cleaned_stop_words'])


In [109]:
posts

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 3, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 4, 0, 2]])

In [27]:
# transformed_posts = np.array([row.toarray() for row in posts])

In [32]:
# transformed_posts.shape=(100,35680)

In [33]:
del df,posts

NameError: name 'df' is not defined

In [112]:
transformed_posts=pd.DataFrame(posts)

In [113]:
X=pd.concat([X,transformed_posts],axis=1)

In [114]:
del transformed_posts

In [116]:
X=X.drop(['cleaned_stop_words'], axis=1)

In [117]:
X

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,15,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,24,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,0,0,0,0,0,0,0,0,0,...,0,0,9,17,4,25,0,14,2,3
14596,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14597,16,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14598,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [118]:
X['age']=X['age'].astype(int)

In [119]:
from scipy import stats

In [120]:
t1=time.time()

X['age'] = stats.zscore(X['age'])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score")

time :::  0.001356363296508789  seconds to find z score


In [121]:
X.head()

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,-1.002358,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,0.252330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,0.126861,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0


,age,category,posts
0,15,Student,"i know, i need to be getting sleep; i know, i..."
1,25,indUnk,love studying?can't get enough of those fat b...
2,17,Education,Im currently updating my site. And almost doin...
3,24,Student,So I was a witness to a pretty nasty accident ...
4,17,indUnk,ok so we skipped multimedia computer lecture...


In [ ]:
del X, X2

In [ ]:
df.head()

In [ ]:
X=df[['cleaned_stop_words']]

In [ ]:
X

In [49]:
X.dtypes

age                 int64
Advertising         uint8
Arts                uint8
Banking             uint8
BusinessServices    uint8
                    ...  
35675               int64
35676               int64
35677               int64
35678               int64
35679               int64
Length: 35700, dtype: object

In [ ]:
dummy_variable_1 = pd.get_dummies(df["category"])
df=pd.concat([df,dummy_variable_1], axis=1)

In [ ]:
df.columns

In [ ]:
df_2=pd.concat([df['age'],df['posts']], axis=1)

In [ ]:
i=0
for x in df.columns:
    i+=1
    if i>7:
        df_2=pd.concat([df_2,df[x]], axis=1)

In [ ]:
df_2.tail()

In [ ]:
columns=df_2.columns[2:]

In [ ]:
df_2.head()

In [ ]:
X=df_2['posts']

In [ ]:
X2.shape

In [ ]:
pd.concat([X, X2], join="inner")

In [ ]:
X.shape

In [ ]:
X2=pd.concat([df_2['posts'],df_2[columns]], axis=1)

In [ ]:
X2.dtypes

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
# vectorizer.fit(X['posts'])
# X['posts'] = vectorizer.transform(X['posts'])
transformed_posts = np.array([row.toarray() for row in vectorizer.fit_transform(X['posts'])])

In [33]:
vectorizer = CountVectorizer()
vectorizer.fit(X['category'])
X['category'] = vectorizer.transform(X['category'])


/home/ebryx/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
import numpy as np

In [22]:
vectorizer = CountVectorizer()
# vectorizer.fit(X['category'])
# X['category'] = vectorizer.transform(X['category'])
transformed_data = np.array([row.toarray() for row in vectorizer.fit_transform(X['category'])])

In [23]:
transformed_data.shape=(14600,45)

In [24]:
transformed_data=pd.DataFrame(transformed_data)

In [25]:
X2=pd.concat([X,transformed_data],axis=1)

In [26]:
X2

,age,category,posts,0,1,2,3,4,5,6,...,35,36,37,38,39,40,41,42,43,44
0,15,Student,"i know, i need to be getting sleep; i know, i...",0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,25,indUnk,love studying?can't get enough of those fat b...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,Education,Im currently updating my site. And almost doin...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24,Student,So I was a witness to a pretty nasty accident ...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,17,indUnk,ok so we skipped multimedia computer lecture...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,Student,ok we got webspace off abp for bgs but of cour...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14596,15,indUnk,"dearest diary, i do not know wad to put for t...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14597,16,indUnk,Good friends are the greatest treasures of th...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14598,23,Student,WE BRING THIS SPECIAL ANNOUCEMENT TO YOU LIVE...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_2.columns

In [ ]:
df_2.head()

In [123]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [127]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [131]:
trainX.shape

(11680, 1041)

In [ ]:
# X=X.transpose()

In [128]:
labels = mlb.fit_transform(labels)

In [129]:
y=labels

In [130]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

In [132]:
X[0].shape

(14600,)

In [133]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [134]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))



In [135]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 20)                20840     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 14)                14350     
Total params: 50,760
Trainable params: 48,712
Non-trainable params: 2,048
_________________________________________________________________


In [136]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)



Train on 11680 samples, validate on 2920 samples
Epoch 1/5
11680/11680 [==============================] - 6s 526us/step - loss: 0.3884 - acc: 0.8571 - val_loss: 0.5929 - val_acc: 0.8571
Epoch 2/5
11680/11680 [==============================] - 5s 443us/step - loss: 0.3807 - acc: 0.8574 - val_loss: 0.8168 - val_acc: 0.8591
Epoch 3/5
11680/11680 [==============================] - 6s 497us/step - loss: 0.3802 - acc: 0.8579 - val_loss: 0.4551 - val_acc: 0.8574
Epoch 4/5
11680/11680 [==============================] - 5s 431us/step - loss: 0.3792 - acc: 0.8586 - val_loss: 0.4316 - val_acc: 0.8576
Epoch 5/5
11680/11680 [==============================] - 5s 429us/step - loss: 0.3780 - acc: 0.8600 - val_loss: 0.6257 - val_acc: 0.7997


In [142]:
testX.shape

(2920, 1041)

In [144]:
testX[2].shape

(2920,)

In [137]:
model.predict(testX[1])

ValueError: Error when checking input: expected dense_5_input to have shape (1041,) but got array with shape (1,)

In [143]:
prediction=model.predict(testX)

In [82]:
for i in trainY:
    print(i)

[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 1 0 0 0 1 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0

### Word2 Vector

In [ ]:
words_of_posts=[]
for j in range(len(data)):
    posts=data[j]['posts']
    
    words=[]
    for i in range(len(posts)):
        words.append(word_tokenize(posts[i]))
    words_of_posts.append(words)

In [ ]:
remove_stop_words(data[1]['posts'])

In [ ]:
if len(data)==len(words_of_posts):
    for i in range(len(data)):
        data[i]['tokenized']=words_of_posts[i]

In [ ]:
df=pd.DataFrame(data)

In [ ]:
X=df['tokenized'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
y=data['horoscope']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y_transform=le.transform(y)

In [ ]:
from keras.utils import np_utils

In [ ]:
dummy_y = np_utils.to_categorical(y_transform)

In [ ]:
#Split

In [ ]:
# model

In [ ]:
# compile

In [ ]:
# fit

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
freq_matrix=_create_frequency_matrix(data[1]['posts'])

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
tf_matrix=_create_tf_matrix(freq_matrix)

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
doc_per_word=_create_documents_per_words(freq_matrix)

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
_create_idf_matrix(freq_matrix,doc_per_word,total_documents)